# Time Series Analysis Examples
In this notebook we will be transforming and exploring time-series data from Onboard API. Make sure to run the following chunk of code first in order to install it.

How to query time-series from data points, process and visually explore them. This notebook is outlined in several YouTube videos that discuss:
- how [to load and filter time-series data](https://youtu.be/KDThdyes8kc) from sensors
- [resample and transform time-series data](https://youtu.be/VlQNGwiSSKM)
- and [create heat maps and boxplots of data](https://youtu.be/QMr0y9LOlr8) for exploration

In [1]:
import pandas as pd
from onboard.client import RtemClient

# Create the client

For this notebook, you need to use the API Key with acces to the general scope and building:read scope. Let's create the client:

In [2]:
api_key = 'ob-p-_KnPccpVFOwKvS5u_SpKixHjKKq6H1JvrzwjxISOfPiNKKZ56_EeE5ORVJOGGtx0d_k'

In [3]:
client = RtemClient(api_key=api_key)

# Load & filter data

We will be using the metadata file we obtained from Onboard API in the previous notebook. Of course you can query these points again, but let's learn how to load and filter the data using *pandas*:

In [4]:
metadata = pd.read_csv("./rtem_all_points_metadata.csv")

Let's check the data types:

In [5]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44390 entries, 0 to 44389
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_point          44390 non-null  int64  
 1   building_id       44390 non-null  int64  
 2   last_updated      44342 non-null  float64
 3   first_updated     44342 non-null  float64
 4   device            0 non-null      float64
 5   objectId          0 non-null      float64
 6   name_point        14658 non-null  object 
 7   description       31843 non-null  object 
 8   units             31687 non-null  object 
 9   tagged_units      14654 non-null  object 
 10  raw_unit_id       44390 non-null  int64  
 11  value             44274 non-null  object 
 12  type              44390 non-null  object 
 13  point_type_id     44390 non-null  int64  
 14  measurement_id    44390 non-null  int64  
 15  datasource_hash   44390 non-null  object 
 16  topic             44390 non-null  object

In [6]:
metadata.head()

,id_point,building_id,last_updated,first_updated,device,objectId,name_point,description,units,tagged_units,...,measurement_id,datasource_hash,topic,equip_id,id_bdg,name_bdg,equip_type_abbr,equip_type_name,area_served_desc,tags
0,310029,441,1.560571e+12,1.550542e+12,NaN,NaN,245 E 149th St. Boiler-1 Fault Code,NaN,?,?,...,18,b115b5053493748f7076ad09f69c4456,nyserda+0000088737+boiler-1+245 E 149th St. Bo...,28797,441,88737,BLR,Boiler,NaN,"['boiler', 'hvac']"
1,310009,441,1.560485e+12,1.550701e+12,NaN,NaN,225 E 149th St. Boiler-1 Exhaust Temp,NaN,F,f,...,1,4a1a11502acee3078352da312c40c262,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,441,88737,BLR,Boiler,NaN,"['boiler', 'hvac']"
2,310010,441,1.560571e+12,1.550504e+12,NaN,NaN,225 E 149th St. Boiler-1 Fault Code,NaN,?,?,...,18,55882633fe8fb5aac7dac643c99b1af5,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,441,88737,BLR,Boiler,NaN,"['boiler', 'hvac']"
3,310011,441,1.560485e+12,1.550504e+12,NaN,NaN,225 E 149th St. Boiler-1 FFWD Temp,NaN,?,?,...,18,831a8df09b848f53e807fa0204c45eaa,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,441,88737,BLR,Boiler,NaN,"['boiler', 'hvac']"
4,310012,441,1.560485e+12,1.550504e+12,NaN,NaN,225 E 149th St. Boiler-1 Fire Rate In,NaN,?,?,...,18,aa147c829c7ed55b6e93cdbbb8b43d55,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,28797,441,88737,BLR,Boiler,NaN,"['boiler', 'hvac']"


We have a lot of points here:

In [7]:
len(metadata.id_point)

44390

Let's filter out only the Zone Temperature sensors -that's the `type_id` 77, you can confirm this querying the ontology data, like we did in the previous notebook ;)- and that are Fan Coil Units:

In [8]:
# boiler_data = metadata[(metadata.equip_type_name == "Boiler")]
# pump_data = metadata[(metadata.equip_type_name == "Pump")]
# hws_data = metadata[(metadata.equip_type_name == "Hot Water Plant")]

In [9]:
ahu_data = metadata[(metadata.equip_type_name == "Air Handling Unit")]

In [10]:
ahu_data.head()
ahu_data.to_csv('sample.csv')

In [11]:
ahu_data['equip_id'].unique()

array([18092, 18093, 18094, ..., 22747, 32352, 32353], dtype=int64)

In [12]:
ahu_sp_data = ahu_data[(ahu_data.type == "Supply Air Temperature Setpoint")]
ahu_sp_data

,id_point,building_id,last_updated,first_updated,device,objectId,name_point,description,units,tagged_units,...,measurement_id,datasource_hash,topic,equip_id,id_bdg,name_bdg,equip_type_abbr,equip_type_name,area_served_desc,tags
1299,198161,140,1.551416e+12,1.546319e+12,NaN,NaN,NaN,UPCM-1 AH-1 Gallery_Supply Air Temp Setpoint,F,NaN,...,1,2f3e73e20430990e1b08249fd50130fe,nyserda+0000083483+ahu1-gallery+UPCM-1 AH-1 Ga...,18092,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1349,198199,140,1.551416e+12,1.509509e+12,NaN,NaN,NaN,AH-1 Lobby_Supply Air Temp Setpoint,F,NaN,...,1,d186863edb7da71e6f32c62b4a745003,nyserda+0000083483+ahu1-lobby+AH-1 Lobby_Suppl...,18093,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1406,198249,140,1.551416e+12,1.536258e+12,NaN,NaN,NaN,UPCM-2 AH-2 2nd Floor_Supply Air Temp Setpoint,F,NaN,...,1,fa697155ce3f5d6ef808b138048b114c,nyserda+0000083483+ahu2-floor2+UPCM-2 AH-2 2nd...,18094,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1438,198287,140,1.551416e+12,1.536592e+12,NaN,NaN,NaN,AHU-2-6TH-FLR_Supply Air Temp Setpoint,F,NaN,...,1,cc3a8ad4c93f7e262f376e17e6fa1ca1,nyserda+0000083483+ahu2-floor6+AHU-2-6TH-FLR_S...,18095,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1947,210916,271,1.551416e+12,1.527826e+12,NaN,NaN,NaN,3RD FL CSC_Supply Air Temp Setpoint,F,NaN,...,1,eeda1eb6df92ae94d094aaaf04116479,nyserda+0000080324+csc3+3RD FL CSC_Supply Air ...,20109,271,80324,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44281,231050,398,1.577768e+12,1.554845e+12,NaN,NaN,NaN,AC-9-1_SaTmpRstSptHiLmt,F,NaN,...,1,bcc97cc43d66460f7c96c8d2b18f268d,nyserda+0000109188+ac9-1+AC-9-1_SaTmpRstSptHiLmt,22736,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44312,231062,398,1.577768e+12,1.571890e+12,NaN,NaN,NaN,AHU_26_02_SaTmpSpt,F,NaN,...,1,7e0987d04c164804b2511221e75e4c64,nyserda+0000109188+ahu26-02+AHU_26_02_SaTmpSpt,22737,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44317,231072,398,1.577768e+12,1.571890e+12,NaN,NaN,NaN,AHU_27_02_SaTmpSpt,F,NaN,...,1,37c8b39c91ed6ebd8ef54ed1b410aeed,nyserda+0000109188+ahu27-02+AHU_27_02_SaTmpSpt,22738,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44351,231105,398,1.577768e+12,1.571890e+12,NaN,NaN,NaN,CRAC-23-1_Discharge Air Temperature Low Limit ...,F,NaN,...,1,917690b8b8adb41a9d0835adfe5e1af1,nyserda+0000109188+crac23-1+CRAC-23-1_Discharg...,22746,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"


In [13]:
ahu_sat_data = ahu_data[(ahu_data.type == "Supply Air Temperature")]
ahu_sat_data

,id_point,building_id,last_updated,first_updated,device,objectId,name_point,description,units,tagged_units,...,measurement_id,datasource_hash,topic,equip_id,id_bdg,name_bdg,equip_type_abbr,equip_type_name,area_served_desc,tags
1302,198159,140,1.551416e+12,1.518575e+12,NaN,NaN,NaN,UPCM-1 AH-1 Gallery_Supply Air Temp,F,NaN,...,1,987bcb47ca9e78bea7aeca833cecef8b,nyserda+0000083483+ahu1-gallery+UPCM-1 AH-1 Ga...,18092,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1352,198197,140,1.551416e+12,1.509509e+12,NaN,NaN,NaN,AH-1 Lobby_Supply Air Temp,F,NaN,...,1,f399cb123ef54820ca9de858f3fca923,nyserda+0000083483+ahu1-lobby+AH-1 Lobby_Suppl...,18093,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1405,198247,140,1.551416e+12,1.518620e+12,NaN,NaN,NaN,UPCM-2 AH-2 2nd Floor_Supply Air Temp,F,NaN,...,1,4bb044197e0865b5aece28acbfd566d1,nyserda+0000083483+ahu2-floor2+UPCM-2 AH-2 2nd...,18094,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1443,198285,140,1.551416e+12,1.536592e+12,NaN,NaN,NaN,AHU-2-6TH-FLR_Supply Air Temp,F,NaN,...,1,2a80e4185f7b244e78726e0911df7387,nyserda+0000083483+ahu2-floor6+AHU-2-6TH-FLR_S...,18095,140,83483,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1654,203561,191,1.562119e+12,1.530562e+12,NaN,NaN,NaN,RTU-1 Front Supply,F,NaN,...,1,4b60eff756d5f4fdb378969a106d9bad,nyserda+0000094414+rtu1+RTU-1 Front Supply,18921,191,94414,AHU,Air Handling Unit,NaN,"['ahu', 'hvac', 'rtu']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44318,231071,398,1.577768e+12,1.571890e+12,NaN,NaN,NaN,AHU_27_02_SATmp,F,NaN,...,1,d261378623a38bdc52e1972dc0b408a2,nyserda+0000109188+ahu27-02+AHU_27_02_SATmp,22738,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44354,231104,398,1.577768e+12,1.571890e+12,NaN,NaN,NaN,CRAC-23-1_Discharge Air Temperature,F,NaN,...,1,7e2b7c496f2959369a125070ee443bab,nyserda+0000109188+crac23-1+CRAC-23-1_Discharg...,22746,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44362,231112,398,1.577768e+12,1.571890e+12,NaN,NaN,NaN,CRAC 5-1_Discharge Air Temperature,F,NaN,...,1,afc0b54922060467673b7e2fbb600ed9,nyserda+0000109188+crac5-1+CRAC 5-1_Discharge ...,22747,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44372,231121,399,1.596254e+12,1.527826e+12,NaN,NaN,AC_S1_DAT_value,AC_S1_DAT_value,F,f,...,1,2911795ee7842d6a496d517e3a89a248,nyserda+0000097151+ac-fan1-supply+AC_S1_DAT_value,32352,399,97151,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"


In [14]:
ahu_hc_data = ahu_data[(ahu_data.type == "Heating Coil Valve Position")]
ahu_hc_data

,id_point,building_id,last_updated,first_updated,device,objectId,name_point,description,units,tagged_units,...,measurement_id,datasource_hash,topic,equip_id,id_bdg,name_bdg,equip_type_abbr,equip_type_name,area_served_desc,tags
2769,356292,470,1.635532e+12,1.604203e+12,NaN,NaN,AHU-02_Heating Coil Valve Command_A02PV,NaN,NaN,%,...,7,6a858fd7b5a57726bb9294f682c8298e,nyserda+0000164423+ahu-2+AHU-02_Heating Coil V...,31252,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
2788,356227,470,1.635530e+12,1.604203e+12,NaN,NaN,AHU-01_Heating Coil Valve Command_A01PV,NaN,NaN,%,...,7,d2b1eb83c94147cdf989e6441d44b65f,nyserda+0000164423+ahu-1+AHU-01_Heating Coil V...,31253,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
2808,356177,470,1.635537e+12,1.604203e+12,NaN,NaN,AHU-17_Heating Coil Valve Command_A17PV,NaN,NaN,%,...,7,8766695d64e23720dfdf87ce6d4147d9,nyserda+0000164423+ahu-17+AHU-17_Heating Coil ...,31254,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
2846,356200,470,1.635536e+12,1.604203e+12,NaN,NaN,AHU-18_Heating Coil Valve Command_A18PV,NaN,NaN,%,...,7,db896bc29faffb7e299e6fc87c65652e,nyserda+0000164423+ahu-18+AHU-18_Heating Coil ...,31255,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
2871,356245,470,1.635533e+12,1.604203e+12,NaN,NaN,AHU-21_Heating Coil Valve Command_A21PV,NaN,NaN,%,...,7,b95518b0a932bcc487f61c1025dabbee,nyserda+0000164423+ahu-21+AHU-21_Heating Coil ...,31256,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22278,380154,492,1.641513e+12,1.596465e+12,NaN,NaN,ACS 02 Heating Coil Valve Command 1_A02VP1.V,NaN,NaN,%,...,7,f9fdfb84740dfc2cd183bb4e5169386c,nyserda+0000122607+ahu-2+ACS 02 Heating Coil V...,33275,492,122607,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
30283,201659,185,1.572494e+12,1.559362e+12,NaN,NaN,AC_47B_HtgVlv,AC_47B_HtgVlv,%,%,...,7,0f3fcf3af5f33d7fac9c6c02f33d2430,nyserda+0000108594+ac-47b+AC_47B_HtgVlv,34967,185,108594,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44229,230968,398,1.577768e+12,1.554845e+12,NaN,NaN,NaN,AC-25-1_HtgVlvPos,%,NaN,...,7,1dab3aad8b30be11d46de8c2d8da7855,nyserda+0000109188+ac25-1+AC-25-1_HtgVlvPos,22734,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
44262,230999,398,1.577768e+12,1.554845e+12,NaN,NaN,NaN,AC-4-1_HtgVlvPos,%,NaN,...,7,f063272fa934bc125ce66468a7966c78,nyserda+0000109188+ac4-1+AC-4-1_HtgVlvPos,22735,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"


In [15]:
shared_ahu_data = pd.merge(ahu_sp_data,ahu_sat_data, on = "equip_id")
shared_ahu_data = pd.merge(shared_ahu_data,ahu_hc_data, on = "equip_id")

In [16]:
shared_ahu_data

,id_point_x,building_id_x,last_updated_x,first_updated_x,device_x,objectId_x,name_point_x,description_x,units_x,tagged_units_x,...,point_type_id,measurement_id,datasource_hash,topic,id_bdg,name_bdg,equip_type_abbr,equip_type_name,area_served_desc,tags
0,356305,470,1.635529e+12,1.604203e+12,NaN,NaN,AHU-02_Supply Air Temperature Setpoint_A02FSP,NaN,NaN,f,...,258,7,6a858fd7b5a57726bb9294f682c8298e,nyserda+0000164423+ahu-2+AHU-02_Heating Coil V...,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
1,356239,470,1.635534e+12,1.604203e+12,NaN,NaN,AHU-01_Supply Air Temperature Setpoint_A01FSP,NaN,NaN,f,...,258,7,d2b1eb83c94147cdf989e6441d44b65f,nyserda+0000164423+ahu-1+AHU-01_Heating Coil V...,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
2,356194,470,1.635532e+12,1.604203e+12,NaN,NaN,AHU-17_Supply Air Temperature Setpoint_A17FSP,NaN,NaN,f,...,258,7,8766695d64e23720dfdf87ce6d4147d9,nyserda+0000164423+ahu-17+AHU-17_Heating Coil ...,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
3,356201,470,1.635539e+12,1.604203e+12,NaN,NaN,AHU-18_High Supply Air Temperature Setpoint_A1...,NaN,NaN,f,...,258,7,db896bc29faffb7e299e6fc87c65652e,nyserda+0000164423+ahu-18+AHU-18_Heating Coil ...,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
4,356220,470,1.635540e+12,1.604203e+12,NaN,NaN,AHU-18_Supply Air Temperature Setpoint_A18FSP,NaN,NaN,f,...,258,7,db896bc29faffb7e299e6fc87c65652e,nyserda+0000164423+ahu-18+AHU-18_Heating Coil ...,470,164423,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,231019,398,1.577768e+12,1.554845e+12,NaN,NaN,NaN,AC-4-1_SaTmpRstSptHiLmt,F,NaN,...,258,7,f063272fa934bc125ce66468a7966c78,nyserda+0000109188+ac4-1+AC-4-1_HtgVlvPos,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
215,231020,398,1.577768e+12,1.554845e+12,NaN,NaN,NaN,AC-4-1_SaTmpRstSptLoLmt,F,NaN,...,258,7,f063272fa934bc125ce66468a7966c78,nyserda+0000109188+ac4-1+AC-4-1_HtgVlvPos,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
216,231052,398,1.577768e+12,1.554845e+12,NaN,NaN,NaN,AC-9-1_SaTmpSpt,F,NaN,...,258,7,ae5363107fc48841033b2608dbeddd13,nyserda+0000109188+ac9-1+AC-9-1_HtgVlvPos,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"
217,231051,398,1.577768e+12,1.554845e+12,NaN,NaN,NaN,AC-9-1_SaTmpRstSptLoLmt,F,NaN,...,258,7,ae5363107fc48841033b2608dbeddd13,nyserda+0000109188+ac9-1+AC-9-1_HtgVlvPos,398,109188,AHU,Air Handling Unit,NaN,"['ahu', 'hvac']"


In [17]:
shared_ahu_data.to_csv("./shared_sat_sp_hc_data.csv")

# Querying time-series: Boiler Heating Hot Water Supply Temp

Now that we have our metadata filtered it's time to get the time-series of those points. Let's import the libraries we will be using:

In [18]:
from datetime import datetime, timezone, timedelta
import pytz
from onboard.client.models import TimeseriesQuery, PointData
from onboard.client.dataframes import points_df_from_streaming_timeseries

First, create a list with the points to query. We already filtered our dataset, so it's all the points there:

In [19]:
ahu_sp_points = list(ahu_sp_data.id_point)
ahu_sat_points = list(ahu_sat_data.id_point)
ahu_hc_points = list(ahu_hc_data.id_point)

In [20]:
ahu_sp_points

[198161,
 198199,
 198249,
 198287,
 210916,
 210938,
 210961,
 267585,
 356171,
 356151,
 356168,
 356305,
 356239,
 356194,
 356201,
 356220,
 356261,
 356246,
 356270,
 356285,
 356325,
 356349,
 356334,
 356357,
 356372,
 356395,
 356380,
 356402,
 356417,
 344004,
 266685,
 266704,
 266723,
 266748,
 266784,
 266787,
 266818,
 266848,
 266878,
 266894,
 266930,
 266958,
 357608,
 357626,
 357619,
 357637,
 357647,
 357657,
 358190,
 358228,
 358208,
 358247,
 358267,
 358287,
 320503,
 320502,
 320535,
 320544,
 308010,
 307609,
 307622,
 307635,
 307655,
 307674,
 307693,
 307710,
 307716,
 307733,
 307748,
 307753,
 307822,
 307774,
 307790,
 307796,
 307801,
 307844,
 307867,
 307889,
 307905,
 307925,
 307941,
 307957,
 307972,
 307990,
 307996,
 308001,
 308005,
 308016,
 308032,
 308047,
 308061,
 308079,
 308174,
 308155,
 308096,
 308114,
 308131,
 308137,
 308192,
 308211,
 308218,
 308228,
 308260,
 308244,
 308277,
 308284,
 308302,
 308317,
 308340,
 308327,
 308351,
 

Let's inspect the datetime period we are working with; we have to convert the `first_updated` and `last_updated` from unix timestamp to a datetime (like we learned in a previous notebook).

In [21]:
print(ahu_sp_data.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min())
print(ahu_sp_data.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min())

2017-08-01 04:00:00+00:00
2018-06-05 03:45:00+00:00


In [22]:
print(ahu_sat_data.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min())
print(ahu_sat_data.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min())

2017-03-23 19:00:00+00:00
2017-05-02 03:00:00+00:00


In [23]:
print(ahu_hc_data.first_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min())
print(ahu_hc_data.last_updated.apply(lambda x: datetime.fromtimestamp(x/1000, timezone.utc)).min())

2018-10-16 04:00:00+00:00
2019-05-15 03:55:00+00:00


Ok, 2017-2018. Let's select a year of data; we are using the [python library datetime](https://docs.python.org/3/library/datetime.html) to create datetime objects. Remember that all the data from the API is in UTC, you have to localize all your datetimes using [PyTZ](https://pythonhosted.org/pytz/) a library designed for that purpose.

In [24]:
# Select your timezone
tz = pytz.timezone('UTC')

# Select your date and time (time is optional)
start = datetime(2018,12,1,0,0,0).replace(tzinfo=tz)
end = datetime(2019,3,1,0,0,0).replace(tzinfo=tz)

print(f"from {start} to {end}")

from 2018-12-01 00:00:00+00:00 to 2019-03-01 00:00:00+00:00


And create the query with all the parameters. Here we are saying "give me the readings fron 20 january 2017 to 20 january 2018 of the points in `points`":

Time to get the data!

In [25]:
i = 29
sp_point = shared_ahu_data['id_point_x'][i]
sat_point = shared_ahu_data['id_point_y'][i]
hc_point = shared_ahu_data['id_point'][i]

timeseries_query_sp = TimeseriesQuery(point_ids = [sp_point], start = start, end = end)
timeseries_query_sat = TimeseriesQuery(point_ids = [sat_point], start = start, end = end)
timeseries_query_hc = TimeseriesQuery(point_ids = [hc_point], start = start, end = end)

In [ ]:
# Exceute query (will return and object)
query_results_sp = client.stream_point_timeseries(timeseries_query_sp)
query_results_sat = client.stream_point_timeseries(timeseries_query_sat)
query_results_hc = client.stream_point_timeseries(timeseries_query_hc)

# Convert to dataframe
data_sp = points_df_from_streaming_timeseries(query_results_sp)
data_sat = points_df_from_streaming_timeseries(query_results_sat)
data_hc = points_df_from_streaming_timeseries(query_results_hc)

In [ ]:
data_sp

In [ ]:
data_sat

In [ ]:
data_hc

In [ ]:
shared_ahu_equip = pd.merge(data_sp,data_sat, on = "timestamp")
shared_ahu_equip = pd.merge(shared_ahu_equip,data_hc, on = "timestamp")

Took some time to retreive that data, is a big dataset. Let's export the csv:

In [ ]:
shared_ahu_equip.to_csv("./rtem_ahu_sat_sp_hc_time_series.csv", index=False)

In [ ]:
print(sp_point)
print(sat_point)
print(hc_point)

# Resampling

Let's load the dataset we just exported just to learn one more trick:

In [ ]:
data = pd.read_csv("./rtem_ahu_sat_sp_time_series.csv")
data.rename(columns={'210916':'sp', '210915': 'sat'}, inplace=True)
data.head()

In [ ]:
data.info()

The timestamp there is not a datetime object yet. We could have parsed it as date [using the parameter `parse_dates` while reading](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) the csv or we can do the following:

In [ ]:
data["timestamp"] = pd.to_datetime(data['timestamp'], format='%Y-%m-%dT%H:%M:%S.%f')

In [ ]:
type(data.timestamp[0])

In [ ]:
data.set_index('timestamp', inplace = True)

See? now is a datetime object located in UTC timezone.

One important thing when working with time-series data is knowing how to resample it. Let's inspect our data:

In [ ]:
data.head(10)

In [ ]:
rtem_temps_avg = data.rolling(window = 5).mean()
# rtem_temps_avg = data

rtem_temps_avg.head()

In [ ]:
rtem_temps_avg.describe()

In [ ]:
rtem_temps_avg.plot(figsize=(25,8), subplots=True, title='RTEM TEMPS FOR FAULT CONDITION 3 MEASURE')

In [ ]:
supply_fan_delta_degf = 2.
oat_rat_delta_degf_min = 10.
supply_degf_err_thres = 2.
return_degf_err_thres = 2.
mix_degf_err_thres = 5.
outdoor_degf_err_thres = 5. # local OAT options Vs global OAT sensor
air_flow_percent_thres = .3
vfd_speed_percent_err_thres = .05
vfd_speed_percent_max = .99
duct_static_inches_err_thres = .1
delta_os_max = 7.
mode_delay = 30.
alarm_dely = 30.
test_mode_delay = 120.

In [ ]:
# make an entire column out of these params in the Pandas Dataframe
rtem_temps_avg['supply_degf_err_thres'] = supply_degf_err_thres

In [ ]:
rtem_temps_avg.columns

In [ ]:
# Create a debug dataframe for later
rtem_temps_debug =  rtem_temps_avg.copy()

In [ ]:
def fault_condition_three_(dataframe):
    return (dataframe.mat - dataframe.mix_degf_err_thres > np.maximum((dataframe.rat + dataframe.return_degf_err_thres) , (dataframe.oat + dataframe.outdoor_degf_err_thres)))

In [ ]:
def fault_condition_five_(dataframe):
    return ((dataframe.sat + dataframe.supply_degf_err_thres) <= (dataframe.mat - dataframe.mix_degf_err_thres + supply_fan_delta_degf))

In [ ]:
def fault_condition_seven_(dataframe):
    return ((dataframe.sat < (dataframe.sp - dataframe.supply_degf_err_thres) & (dataframe.hc >= 99))

In [ ]:
import numpy as np

rtem_temps_avg['fc7_flag'] = fault_condition_seven_(rtem_temps_avg)

In [ ]:
rtem_temps_avg['fc7_flag'].value_counts()

In [ ]:
rtem_temps_avg.columns

In [ ]:
# change boolean column to int for plots

rtem_temps_avg['fc7_flag'] = rtem_temps_avg['fc7_flag'].astype(int)

# drop params column for better plot
rtem_temps_avg = rtem_temps_avg.drop(['supply_degf_err_thres'],axis=1)

In [ ]:
fdd_plot = rtem_temps_avg

In [ ]:
fdd_plot.columns

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(25,8))

plot1a = fdd_plot.mat.plot(ax=ax)
plot1b = fdd_plot.sat.plot(ax=ax)
plot1d = fdd_plot.fc5_flag.plot(ax=ax, color='k', secondary_y=True)

ax.set_ylabel('Temperatures')
ax.right_ax.set_ylim(0,1)
plt.ylabel('FDD FLAG')

ax.set_title("RTEM Temp Sensors and FDD 5 Flag")
fig.legend(['sp','sat','hc','fc7_flag'],bbox_to_anchor=(.95, .9))

# defining display layout
plt.tight_layout()

# show plot
plt.show()

# Plotting

## Lineplots

We are going to explore our sensors data with some simple lineplots. We are going to use [*Seaborn*](https://seaborn.pydata.org/) which works on [*Matplotlib*](https://matplotlib.org/stable/users/index.html). Let's import them:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# This is for the visual stype, I like "ggplot"
plt.style.use('ggplot')

# This for the figure size
plt.rcParams["figure.figsize"] = (20,9)

Let's plot the raw data of our sensors:

In [ ]:
# These are the sensors, one per column (first one is the timestamp)
cols = data.columns[1:]
cols

In [ ]:
for col in cols:
    plt.figure()
    sns.lineplot(data=data, x="timestamp", y=col)

We have a huge gap with no data and the rest it's noisy and difficult to see. Let's filter data from september 1st 2017:

In [ ]:
data = data[data.timestamp >= "2017-09-01"]

One useful technique to deal with noisy data is to resample it by hour (or a longer time frequency) aggregating by the mean: this way we can visualize the hourly/daily behaviour more smoothed.

But first, let's go deeper into our `timestamp` column. This is a datetime column, meaning that we have some functions to handle the dates better. With [this pandas method](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.html) we can access our date: we can get the date, the weekday, the hour, the seconds, the minutes; all the information in our date can be obtained.

Let's try an example. This is our raw timestamp:

In [ ]:
data.timestamp

We can get the date:

In [ ]:
data.timestamp.dt.date

Even the he day of the week (with Monday=0, Sunday=6)

In [ ]:
data.timestamp.dt.weekday

The hour

In [ ]:
data.timestamp.dt.hour

Or the minutes

In [ ]:
data.timestamp.dt.minute

Pretty simple! You have all the datetime information handy.

Now let's smooth the data to plot it!

In [ ]:
data_hour = data.resample('1H', on="timestamp").mean()
data_hour.head()

And we have the mean temperature by hour! Plotting it is simple with this data set:

In [ ]:
fig = sns.lineplot(data=data_hour)

# This is just for hiding the legend for now, there are a lot of labels and is difficult to see the plot
plt.legend([],[], frameon=False)

Let's do it again but daily:

In [ ]:
data_day = data.resample('24H', on="timestamp").mean()
data_day.head()

In [ ]:
sns.lineplot(data=data_day)

# This is just for hiding the legend for now, there are a lot of labels and is difficult to see the plot
plt.legend([],[], frameon=False)

## Heatmaps

Lineplots are easy but still pretty diffcult to visualize. Instead, we can use [heatmaps](https://seaborn.pydata.org/generated/seaborn.heatmap.html), it's also really easy. Remember you can transpose your data to change the axis, like this:

In [ ]:
data_hour.T

And super-easy to plot:

In [ ]:
sns.heatmap(data_hour.T, cmap="hot_r")

In [ ]:
sns.heatmap(data_day.T, cmap="hot_r")

With this kind of plot is pretty evident where we have gaps with no-readings.

# Summary stats

Usually is useful to get some summary stats from our data. Fortunately, pandas has a simple method to obtain them:

In [ ]:
data.describe()

These stats are usually visualizes in boxplot and histograms; these two plots can be easily obtained with the plotting libraries we are using. But first I suggest processing a little the dataframe to be able to plot all sensors in the same figure. We are using the pandas method [melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html). We have the data in this way right now:

In [ ]:
data.head()

If we use the `melt` method we can convert our columns in rows, creating a new one that indicates which column it was. Is easier seeing the results, check it out:

In [ ]:
melted_data = data.melt(id_vars="timestamp", var_name="sensor", value_name="temperature")
melted_data

And now that we have the data in this shape, is easy to get a boxplot:

In [ ]:
fig = sns.boxplot(y="temperature", x="sensor", data=melted_data)
labels = plt.xticks(rotation=90)

We can also plot an histogram, but with this number of sensors a boxplot is cleaner:

In [ ]:
fig = sns.histplot(data=melted_data, x="temperature", hue="sensor", bins=20, kde=True, palette="Dark2")

# This is just for hiding the legend for now, there are a lot of labels and is difficult to see the plot
plt.legend([],[], frameon=False)

Finally, let's create a dataset with the stats. 

In [ ]:
stats_data = data.describe()
stats_data

We are transposing it and reseting the index to keep a column with the point id (was the name of the column):

In [ ]:
stats_data = stats_data.T.reset_index().rename(columns={"index":"id_point"})
stats_data

And now let's convert the `id_point` column in our metadata set to string (is an int right now) so we are able to merge with `stats_data`:

In [ ]:
metadata['id_point'] = metadata['id_point'].astype(str)

And merge!

In [ ]:
stats_merged = pd.merge(stats_data, metadata, how="left", on="id_point")
stats_merged.head()

In [ ]:
metadata.T

Finally, plot! we want to visualize the mean temperature of the points, colored by the building name.

In [ ]:
fig = sns.barplot(data=stats_merged, x="id_point", y="mean", hue="name_bdg", dodge=False)
labels = plt.xticks(rotation=90)